## Day 46 Lecture 1 Assignment

In this assignment, we will calculate a distance matrix for geographical Starbucks data and use it to identify locations that are close together and far apart. We will perform clustering on this dataset later on.

We will be using the "haversine" package to compute geographical distance. It can be pip installed.

In [1]:
!pip install haversine

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from haversine import haversine

Below is a convenience function for calculating geographical distance matrices using lat-long data.

In [3]:
def geo_sim_matrix(df, col_name = 'Coordinates'):
    """
    A function that computes a geographical distance matrix (in miles).
    Each row in the dataframe should correspond to one location.
    In addition, the dataframe must have a column containing the lat-long of each location as a tuple (i.e. (lat, long)).
    
    Parameters:
        df (pandas dataframe): an nxm dataframe containing the locations to compute similarities between.
        col (string): the name of the column containing the lat-long tuples.
        
    Returns:
        distance (pandas dataframe): an nxn distance matrix between the geographical coordinates of each location.
    """
    
    df = df.copy()
    df.reset_index(inplace=True)
    haver_vec = np.vectorize(haversine, otypes=[np.float32])
    distance = df.groupby('index').apply(lambda x: pd.Series(haver_vec(df[col_name], x[col_name])))
    distance = distance / 1.609344  # converts to miles
    distance.columns = distance.index
    
    return distance


This dataset contains the latitude and longitude (as well as several other details we will not be using) of every Starbucks in the world as of February 2017. Each row consists of the following features, which are generally self-explanatory:

- Brand
- Store Number
- Store Name
- Ownership Type
- Street Address
- City
- State/Province
- Country
- Postcode
- Phone Number
- Timezone
- Longitude
- Latitude

Load in the dataset.

In [4]:
# answer goes here
df = pd.read_csv('data/starbucks_locations.csv')
df.head()

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude
0,Starbucks,47370-257954,"Meritxell, 96",Licensed,"Av. Meritxell, 96",Andorra la Vella,7,AD,AD500,376818720,GMT+1:00 Europe/Andorra,1.53,42.51
1,Starbucks,22331-212325,Ajman Drive Thru,Licensed,"1 Street 69, Al Jarf",Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.42
2,Starbucks,47089-256771,Dana Mall,Licensed,Sheikh Khalifa Bin Zayed St.,Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.39
3,Starbucks,22126-218024,Twofour 54,Licensed,Al Salam Street,Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.38,24.48
4,Starbucks,17127-178586,Al Ain Tower,Licensed,"Khaldiya Area, Abu Dhabi Island",Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.54,24.51


Begin by narrowing down the dataset to a specific geographic area of interest. Since we will need to manually compute a distance matrix, which will be on the order of $n^{2}$ in terms of size, we would recommend choosing an area with 3000 or less locations. In this example, we will use Hawaii, which has about 100 locations; for reference, California has about 2800 locations. Feel free to choose a different region that is of more interest to you, if desired.

Subset the dataframe to only include records for Starbucks locations in Hawaii.

In [5]:
# answer goes here

df_il = df[df['State/Province']=='IL']

df_il['Country'].nunique()

1

The haversine package takes tuples with 2 numeric elements and interprets them as lat-long to calculate distance, so add a new column called "Coordinates" that converts the lat and long in each row into a tuple. In other words, the last two columns of the dataframe should initially look like this:

**Latitude, Longitude**  
39.14, -121.64  
34.13, -116.40  
...

After adding the new column, the last three columns should look like this:

**Latitude, Longitude, Coordinates**  
39.14, -121.64,   (39.14, -121.64)  
34.13, -116.40,   (34.13, -116.40)  
...

In [6]:
df_il.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 575 entries, 17457 to 18031
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Brand           575 non-null    object 
 1   Store Number    575 non-null    object 
 2   Store Name      575 non-null    object 
 3   Ownership Type  575 non-null    object 
 4   Street Address  575 non-null    object 
 5   City            575 non-null    object 
 6   State/Province  575 non-null    object 
 7   Country         575 non-null    object 
 8   Postcode        575 non-null    object 
 9   Phone Number    560 non-null    object 
 10  Timezone        575 non-null    object 
 11  Longitude       575 non-null    float64
 12  Latitude        575 non-null    float64
dtypes: float64(2), object(11)
memory usage: 62.9+ KB


In [7]:
# answer goes here


df_il['Coordinates']=df_il[['Latitude', 'Longitude']].apply(tuple,1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
df_il.head()

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude,Coordinates
17457,Starbucks,2726-85202,Lake & Rohlwing,Company Owned,"1590 W. Lake St., Suite 101",Addison,IL,US,601011824,630773-4743,GMT-06:00 America/Chicago,-88.03,41.95,"(41.95, -88.03)"
17458,Starbucks,76107-79323,Super Target Algonquin ST-1801,Licensed,"750 Randall Rd, Ryan Place",Algonquin,IL,US,60102,847-458-5340,GMT-06:00 America/Chicago,-88.34,42.17,"(42.17, -88.34)"
17459,Starbucks,2626-63429,East Algonquin & Ryan,Company Owned,"1532 E. Algonquin Road, Floor 1",Algonquin,IL,US,601024519,847-458-6740,GMT-06:00 America/Chicago,-88.27,42.16,"(42.16, -88.27)"
17460,Starbucks,18024-182807,Randall & Harnish,Company Owned,"790 S. Randall Road, Suite 1",Algonquin,IL,US,60102,8474580669,GMT-06:00 America/Chicago,-88.34,42.16,"(42.16, -88.34)"
17461,Starbucks,21358-211207,Arlington Heights & Golf Road,Company Owned,1808 South Arlington Heights Road,Arlington Heights,IL,US,60005,8476402779,GMT-06:00 America/Chicago,-87.98,42.05,"(42.05, -87.98)"


Calculate the distance matrix using the starter code/function geo_sim_matrix() provided earlier in the notebook. It assumes the column containing the coordinates for each location is called "Coordinates". Examine the docstring for more details.

Note: the latitude and longitudes provided only go out to two decimal places, which limits the resolution of the distance calculations to about 0.5 miles. Distances that are very small may not be accurately represented here (e.g. several instances of "0 distance" for distinct Starbucks locations in very close proximity).

In [24]:
# answer goes here

dist_mat = geo_sim_matrix(df_il)



In [31]:
dist_mat

index,17457,17458,17459,17460,17461,17462,17463,17464,17465,17466,...,18022,18023,18024,18025,18026,18027,18028,18029,18030,18031
index,,,,,,,,,,,,,,,,,,,,,
17457,NaN,21.998655,19.029597,21.527960,7.370898,9.341678,9.686688,11.348866,21.130444,18.405280,...,16.516217,16.516217,19.506333,18.539901,13.856968,12.606001,12.479327,32.891659,29.349573,29.109865
17458,21.998655,NaN,3.650891,0.690934,20.229834,19.468171,17.303818,18.904129,26.335757,30.067912,...,30.365815,30.365815,33.256965,31.018190,33.609257,30.922485,31.396745,11.190566,34.509039,35.610755
17459,19.029597,3.650891,NaN,3.585198,16.696038,15.856924,13.693897,15.255140,26.181536,29.037429,...,26.715266,26.715266,29.607542,27.374610,31.320382,28.838915,29.220768,13.863712,34.722152,35.635412
17460,21.527960,0.690934,3.585198,NaN,19.957956,19.260005,17.096893,18.766155,25.647000,29.385448,...,30.234197,30.234197,33.151695,30.935839,33.015211,30.307037,30.790857,11.793137,33.831552,34.927262
17461,7.370898,20.229834,16.696038,19.957956,NaN,2.072803,3.441989,4.145605,27.305780,25.526491,...,11.473452,11.473452,14.624680,12.987062,20.785362,19.891942,19.678132,30.030112,35.905930,35.868051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18027,12.606001,30.922485,28.838915,30.307037,19.891942,21.912949,22.258775,23.942811,16.031961,9.506581,...,26.697002,26.697002,29.164503,28.974314,3.387087,NaN,1.030632,42.098510,21.544009,20.540819
18028,12.479327,31.396745,29.220768,30.790857,19.678132,21.719135,22.163605,23.765642,17.058825,10.515052,...,26.096327,26.096327,28.504740,28.384874,2.481873,1.030632,NaN,42.583567,22.533431,21.502510
18029,32.891659,11.190566,13.863712,11.793137,30.030112,28.833267,26.748247,27.739861,36.107627,40.699050,...,38.730440,38.730440,41.288270,38.874222,44.799738,42.098510,42.583567,NaN,43.540968,44.913657


For each Starbucks, identify its nearest neighboring location in Hawaii (and presumably in the world). Save the output to a dataframe with three columns: Location, Nearest Neighbor, and Distance (Miles).

In [27]:
# answer goes here
np.fill_diagonal(dist_mat.values, np.nan)

df_il['Nearest Neighbor'] = dist_mat.idxmin()
# Find min value for each store
df_il['Distance (Miles)'] = dist_mat.min()

df_il.head()



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude,Coordinates,Nearest Neighbor,Distance (Miles)
17457,Starbucks,2726-85202,Lake & Rohlwing,Company Owned,"1590 W. Lake St., Suite 101",Addison,IL,US,601011824,630773-4743,GMT-06:00 America/Chicago,-88.03,41.95,"(41.95, -88.03)",17797,1.722053
17458,Starbucks,76107-79323,Super Target Algonquin ST-1801,Licensed,"750 Randall Rd, Ryan Place",Algonquin,IL,US,60102,847-458-5340,GMT-06:00 America/Chicago,-88.34,42.17,"(42.17, -88.34)",17460,0.690934
17459,Starbucks,2626-63429,East Algonquin & Ryan,Company Owned,"1532 E. Algonquin Road, Floor 1",Algonquin,IL,US,601024519,847-458-6740,GMT-06:00 America/Chicago,-88.27,42.16,"(42.16, -88.27)",17807,3.368986
17460,Starbucks,18024-182807,Randall & Harnish,Company Owned,"790 S. Randall Road, Suite 1",Algonquin,IL,US,60102,8474580669,GMT-06:00 America/Chicago,-88.34,42.16,"(42.16, -88.34)",17458,0.690934
17461,Starbucks,21358-211207,Arlington Heights & Golf Road,Company Owned,1808 South Arlington Heights Road,Arlington Heights,IL,US,60005,8476402779,GMT-06:00 America/Chicago,-87.98,42.05,"(42.05, -87.98)",17934,1.026121


If the nearest neighbor of a Starbucks location is far away, we could consider that Starbucks to be "on an island". Which five Starbucks in Hawaii are the most "on an island"?

In [30]:
# answer goes here

df_il.sort_values(by='Distance (Miles)', ascending=False).head(5)



,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude,Coordinates,Nearest Neighbor,Distance (Miles)
17845,Starbucks,48900-254804,Kroger Mt. Vernon 711 - Venue,Licensed,415 SOUTH 42ND STREET,MT. VERNON,IL,US,62864,618-244-6210,GMT-06:00 America/Chicago,-88.94,38.31,"(38.31, -88.94)",17828,38.753508
17827,Starbucks,26714-244394,Hy-Vee Macomb #1420,Licensed,1602 East Jackson,Macomb,IL,US,61455,3098379917,GMT-06:00 America/Chicago,-90.65,40.46,"(40.46, -90.65)",17505,32.930632
17515,Starbucks,13613-106969,Charleston-Lincoln & E St,Company Owned,437 W Lincoln Avenue,Charleston,IL,US,619203022,217 345 5553,GMT-06:00 America/Chicago,-88.19,39.48,"(39.48, -88.19)",17728,31.057108
17728,Starbucks,9648-96552,I-70 & Hwy 32,Company Owned,1700 Avenue of Mid America,Effingham,IL,US,624014500,217-342-5577,GMT-06:00 America/Chicago,-88.57,39.14,"(39.14, -88.57)",17515,31.057108
17754,Starbucks,24096-231765,Hy-Vee @ Galesburg #2,Licensed,"2030 E Main St, Geneva Commons Shopping Center",Galesburg,IL,US,61401,309-342-1615,GMT-06:00 America/Chicago,-90.34,40.95,"(40.95, -90.34)",17505,30.591711
